In [1]:
import pandas as pd
from src.smiles_processor import DataFrameSmilesProcessor, save_processed_df
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.error')
RDLogger.DisableLog('rdApp.info')
RDLogger.DisableLog('rdApp.warning')

### Dataset specific pre-processing

In [2]:
stokes_2020 = pd.read_csv('data/raw/stokes_2020.csv')
stokes_2020['target'] = 'Escherichia coli BW25113'
stokes_2020['antimicrobial_activity'] = stokes_2020['Activity'].apply(lambda x: 1 if x == 'Active' else 0)
stokes_2020.drop(columns=['Name', 'Activity'], inplace=True)

swanson_2024 = pd.read_csv('data/raw/swanson_2024.csv')
swanson_2024['target'] = 'Acinetobacter baumannii ATCC 17978'
swanson_2024.rename(columns={'smiles': 'SMILES', 'antibiotic_activity': 'antimicrobial_activity'}, inplace=True)
swanson_2024.drop(columns=['clogp_6.5', 'mol_weight', 'clogp'], inplace=True)

swanson_2025 = pd.read_csv('data/raw/swanson_2025.csv')
swanson_2025['target'] = 'Staphylococcus aureus RN4220'
swanson_2025.rename(columns={'smiles': 'SMILES', 's_aureus_activity': 'antimicrobial_activity_continuous', 's_aureus_activity_true': 'antimicrobial_activity'}, inplace=True)

liu_2023 = pd.read_csv('data/raw/liu_2023.csv')
liu_2023['target'] = 'Acinetobacter baumannii ATCC 17978'
mean_liu = liu_2023['Mean'].mean()
std_liu = liu_2023['Mean'].std()
threshold_liu = mean_liu - std_liu
liu_2023['antimicrobial_activity'] = (liu_2023['Mean'] < threshold_liu).astype(int)
liu_2023.drop(columns=['Name', 'Rep 1', 'Rep 2'], inplace=True)


wong_2024 = pd.read_csv('data/raw/wong_2024.csv')
wong_2024['antimicrobial_activity'] = 0
wong_2024.loc[:511, 'antimicrobial_activity'] = 1
wong_2024['target'] = 'Staphylococcus aureus RN4220'
wong_2024.drop(columns=['R1_50uM', 'R2_50uM'], inplace=True)


co_add = pd.read_csv('data/raw/CO_ADD.csv')
co_add = co_add[co_add['CONC'] == '32 ug/mL']
co_add = co_add[~co_add['STRAIN'].isin(['lpxC; MB4902', 'tolC; MB5747'])]
co_add = co_add[co_add['INHIB_STD'] <= 15]
co_add.drop(columns=['COMPOUND_NAME','Unnamed: 0', 'CONC', 'INHIB_STD', 'COMPOUND_CODE', 'COADD_ID','PROJECT_ID', 'LIBRARY_NAME', 'ASSAY_ID','NASSAYS','STRAIN'], inplace=True)
co_add_by_organism = {org: group.copy() for org, group in co_add.groupby('ORGANISM')}


/tmp/ipykernel_79046/410817397.py:31: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  co_add = pd.read_csv('data/raw/CO_ADD.csv')


### SMILES preparation
Basically, RDKit first runs sanitization (Mol From Smiles), checking if the molecule is feasible.
Then, we run some standardization with `rdMolStandardize`, consisting of CleanUp (removing Hs, disconnecting metals, etc), Uncharging, and getting the (usually) most stable tautomer with TautomerEnumeration. (Ref: [RDKit](https://www.rdkit.org/docs/RDKit_Book.html#molecular-sanitization) and [a short post about it](https://www.blopig.com/blog/2022/05/molecular-standardization/))


In [3]:
processor = DataFrameSmilesProcessor()

In [4]:
for keys, df in co_add_by_organism.items():
    processed_df = processor.process(df, 'SMILES')
    save_processed_df(processed_df, f"co_add_{keys.replace(' ', '_').replace('/', '_')}")

2025-10-15 12:50:37.902 | INFO     | src.smiles_processor:process:18 - Starting SMILES validation of 65441 entries.
2025-10-15 12:50:44.390 | INFO     | src.smiles_processor:process:23 - Number of invalid SMILES: 4
2025-10-15 12:50:44.393 | SUCCESS  | src.smiles_processor:process:27 - SMILES validation completed. 65437 valid entries retained.
2025-10-15 12:50:44.393 | INFO     | src.smiles_processor:process:30 - Starting SMILES standardization of 65437 entries.
2025-10-15 12:51:13.513 | ERROR    | src.utils.smiles_utils:standardize_smiles:62 - Standardization failed for SMILES: c(OCO1)(c1c2OC)c(OC)cc2\C=C\[N+](=O)[O-] | Error: Invariant Violation
	could not find atom2
	Violation occurred on line 223 in file Code/GraphMol/Canon.cpp
	Failed Expression: firstFromAtom2
	RDKIT: 2024.03.3
	BOOST: 1_85

2025-10-15 12:51:19.375 | ERROR    | src.utils.smiles_utils:standardize_smiles:62 - Standardization failed for SMILES: n(C)(c1c(c2\C=C\[N+](=O)[O-])cccc1)c2 | Error: Invariant Violation
	could

In [5]:
stokes_2020 = processor.process(stokes_2020, 'SMILES')
swanson_2024 = processor.process(swanson_2024, 'SMILES')
swanson_2025 = processor.process(swanson_2025, 'SMILES')
liu_2023 = processor.process(liu_2023, 'SMILES')
wong_2024 = processor.process(wong_2024, 'SMILES')

2025-10-15 13:43:29.586 | INFO     | src.smiles_processor:process:18 - Starting SMILES validation of 2335 entries.
2025-10-15 13:43:29.785 | INFO     | src.smiles_processor:process:23 - Number of invalid SMILES: 0
2025-10-15 13:43:29.785 | SUCCESS  | src.smiles_processor:process:27 - SMILES validation completed. 2335 valid entries retained.
2025-10-15 13:43:29.786 | INFO     | src.smiles_processor:process:30 - Starting SMILES standardization of 2335 entries.
2025-10-15 13:44:09.471 | INFO     | src.smiles_processor:process:37 - Number of invalid SMILES after standardization: 0
2025-10-15 13:44:09.472 | SUCCESS  | src.smiles_processor:process:41 - SMILES standardization completed. 2335 valid entries retained.
2025-10-15 13:44:09.473 | INFO     | src.smiles_processor:process:46 - Removing duplicates based on standardized SMILES.
2025-10-15 13:44:09.474 | SUCCESS  | src.smiles_processor:process:48 - Duplicate removal completed. 2291 unique entries retained, from 2335 original entries.
202

In [6]:
for df, name in zip([stokes_2020, swanson_2024, swanson_2025, liu_2023, wong_2024],
                    ['stokes_2020', 'swanson_2024', 'swanson_2025', 'liu_2023', 'wong_2024']):
    save_processed_df(df, name)